# Exploratory doc2vec


In [3]:
import os
import gensim
import numpy as np
import scipy as sp
from smart_open import smart_open
import multiprocessing

cores = multiprocessing.cpu_count()
usecores=np.int(3*cores/4)




Get and process stopwords. List has been gatherd from various sources and expanded to include (some) utterances such as uhm etc.

In [7]:
with open('/Users/eslt0101/Projects/EviDENce/ML/stopwords-nl-stopwords-iso_expanded.txt','r') as stopwords_in:
    dutch_stopwords =stopwords_in.readlines()
    for i,sw in enumerate(dutch_stopwords):
        dutch_stopwords[i] = sw.replace('\n','')
        
    dutch_stopwords = gensim.utils.simple_preprocess(' '.join(dutch_stopwords),deacc=True)
    #print(dutch_stopwords)

### Define utility functions

In [5]:
def read_corpus(corpus_token_file,labeled=False):
    with smart_open(corpus_token_file,'r') as tf:
        for i,text_line in enumerate(tf):
            if labeled :
                #yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text_line), [fn_line.rstrip()])
                yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords], [i])
            else:
                yield [token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords]

This (below) reads in the corpus in unprocessed fashin allowing inspection for relevance 

In [6]:
def read_corpus_lookup(corpus_filenames_file, corpus_token_file) :
    with smart_open(corpus_filenames_file, 'r') as fnf, smart_open(corpus_token_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1
        

In [8]:
corpus_dir = '/Users/eslt0101/Data/eScience/EviDENce/Data/NR-Teksts/EviDENce_NR_output/TargetSize150'
#corpus_text_file = 'test_GV_corpus_terms_para.txt'
#corpus_filenames = 'test_GV_corpus_terms_para_filenames.txt'
corpus_token_file = 'GV_corpus_terms_para_150.txt'
corpus_text_file = 'GV_corpus_terms_para_150.txt'
corpus_filenames = 'GV_corpus_terms_para_150_filenames.txt'
full_ctokf = os.path.join(corpus_dir,corpus_token_file)
full_ctf = os.path.join(corpus_dir,corpus_text_file)
full_cff = os.path.join(corpus_dir,corpus_filenames) 

In [10]:
corp = list(read_corpus(full_ctokf,labeled=True))

In [11]:
corp_lookup = list(read_corpus_lookup(full_cff,full_ctokf))

In [1659]:
#model = gensim.models.doc2vec.Doc2Vec(vector_size=50, negative=15, hs=0, min_count=4, epochs=30, workers=usecores)

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=30)#, workers=usecores)



In [1660]:
model.build_vocab(corp)

In [1662]:
%time model.train(corp, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1min 24s, sys: 8.29 s, total: 1min 32s
Wall time: 40.4 s


load or save as desired

In [14]:
#model.save('/Users/eslt0101/Projects/EviDENce/ML/model_default_v50_mc2_e30_freeze.d2v')
model = gensim.utils.SaveLoad.load('/Users/eslt0101/Projects/EviDENce/ML/model_default_v50_mc2_e30_freeze.d2v')




This is an initial test of retrieval on the full corpus

In [15]:
ranks = []
first_rank =[]
second_ranks = []
least_ranks =[]
doc_id =0
#for i in range(len(corp)):
for i in range(100):
    inferred_vector = model.infer_vector(corp[i].words)
    inferred_vector_tag = corp[i].tags
    print('actual snippet')
    print(corp[i].words)
    print(corp[i].tags)
    print("-------------")
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [tag for tag,sim in sims].index(inferred_vector_tag)
    ranks.append(rank)
    second_ranks.append(sims[1])
    least_ranks.append(sims[-1])
    print(corp_lookup[sims[0][0]])
    print("++++")
    print(rank)
    print(corp_lookup[inferred_vector_tag[0]])
    #print(sims[1])
    #print(sims[2])
    #print(sims[-1])
    print('=======================')
    print
#sims[0]
    #rank = [tag for tag, sim in sims].index(inferred_vector_tag)
    #ranks.append(rank)
    #second_ranks.append(sims[1])
    #least_ranks.append[sims[-1]]

actual snippet
['werk', 'ogenblik', 'staking', 'uitbrak', 'staakten', 'eisten', 'fabriek', 'leiding', 'verwalter', 'neeee', 'jood', 'joodse', 'man', 'christenman', 'fabriek', 'leidde', 'eiste', 'heette', 'vak', 'riem', 'afgooien', 'meestaakte', 'protesteerde', 'heette', 'riem', 'afgooide', 'riem', 'natuurlijk', 'afgooien', 'eisten', 'fabrikant', 'directeur', 'zeggen', 'waarnemer', 'eiste', 'ie', 'ie', 'fabriek', 'bleef', 'draaien', 'bleven', 'werken']
[0]
-------------
([22089], ['GV_AVA_doven_01_conversation_clipped_150_paragraph_370-374_text.txt'], ["En ' s avonds moesten we weer klaar staan voor appel , werd iedereen weer geteld , en dan kregen we eten , kregen we een klein stukje brood , soep , en dan kon u vier uur slapen , we waren zo verschrikkelijk moe , met elkaar praten , kon u niet , we waren moe en we waren verdrietig , en we hadden geen leven meer , toen . Eigenlijk wilde ik het liefst maar dood , voor mij was dit geen leven , want zoveel verdriet , maar niet alleen ik had

([8], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_227-234_text.txt'], ["Ja . Mise . Ja , ja . Eh , hoorden onderling wie we waren en zo verbeeld ik me heel sterk dat ik in de , dat ik bij hun ook zat van eh van Veffer met z' n met z' n kinderen , d' r waren verschillende waren d' r wel vrijwillig meegegaan . En ik weet niet in hoeverre zij eh meegegaan waren , of die vader [ ja ] gehaald werd of dat dat de dochter gehaald werd . Dat weet ik niet meer . Wat trof u aan in de Hollandsche Schouwburg ? Nou , in de Hollandsche Schouwburg was het heel erg druk en benauwd en eh de en eh , ze waren we niet zo lang maar daar zaten we , je kon je plaats uitkiezen . Toen kwam die commandant , eh , ik weet z' n naam niet meer . Die kwam op het eh op ' t toneel en die eh sprak enige hatelijke woorden en eh , maar in Westerbork kregen we een pakje brood . \n"])
++++
0
([8], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_227-234_text.txt'], ["Ja . Mise . Ja , ja . 

([13], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_660-673_text.txt'], ["Jaren later heb ik dat in Auschwitz in het eh in de hoe heet dat , in eh in het archief [ humum ] heb ik dat opgevraagd en heb ik dat gekregen . Is zij overleden in Auschwitz toen u daar nog werkte ? In Birkenau . In Birkenau , ja . Ja , toen Ja . Jaa , in eh in een van de latere maanden , zo in eh oktober , november of zoiets . Had u enig eh idee wat er verder in Auschwitz eh aan industrie bestond ? Jawel . D' r was een eh grote fabriek voor de eh hoe heette dat , eh , debudie ? Ik bedoel eigenlijk metaal Was er was er was waren dat Maar ik bedoelde eigenijk ironisch de vernietigings eh barakken , de gaskamers . Nou , d' r waren wel die er heel dichtbij waren . Ja . Maar wist u wat er gebeurde in Auschwitz verder ? \n"])
++++
0
([13], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_660-673_text.txt'], ["Jaren later heb ik dat in Auschwitz in het eh in de hoe heet dat , in eh in

actual snippet
['ondervond', 'amsterdam', 'kwam', 'terug', 'kwam', 'kennis', 'vertelde', 'overleden', 'vrouw', 'meegemaakt', 'beth', 'sjalom', 'getroffen', 'doof', 'moeilijk', 'praten', 'overleefd', 'meegemaakt', 'vrouw', 'vertelde', 'vriendin', 'vertellen', 'trof', 'straat', 'ogenblik', 'hield', 'staande', 'horen', 'doorgelopen', 'wou', 'vertellen', 'idee']
[18]
-------------
([18], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_770-775_text.txt'], ["Nou , dat eh ondervond ik wel . In Amsterdam. [ humum ] Want zij kwam niet terug . En er kwam wel een kennis die vertelde hoe ze overleden is. [ humum ] Een vrouw die haar heeft meegemaakt en die die ik in Beth Sjalom dan ook getroffen heb. [ humum ] Hè , was doof dus die kon moeilijk praten , maar die had het , ja die had het ook overleefd. [ humum ] En die had ' r meegemaakt . En die andere vrouw waar ik van vertelde die vriendin , die heeft eh nooit wat willen vertellen . En die trof ik op straat op een gegeven ogenbli

([25], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_561-570_text.txt'], ["Dat deed u natuurlijk zeer . Ja , vanzelf. [ hum ] Je gaat daar geen grote verwachtingen van , inderdaad gingen dat kind en moeder gingen direct naar de naar eh naar naar de gaskamer , in Auschwitz . En van die vader heb ik ook nooit meer wat gehoord . Ik ga even terug naar eh de treinreis . Heeft u , heeft u het gevoel dat ooit met eh uw vrouw Sophie eh in het reine te kunnen brengen ? Of er in het reine gebracht te hebben ? Is het uitgesproken ? Jaaa , ja dat wel . Dat heeft ze zelf wel begrepen . Want ik zei , zo eh banaal eh , als we terugkomen krijgt ie eh , krijgt ie broertjes en zussies . Dus . Met dat beeld zat je met z' n tweeën in de in de trein . Nou ja , maar ze zal het wel begrepen hebben . \n"])
++++
0
([25], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_561-570_text.txt'], ["Dat deed u natuurlijk zeer . Ja , vanzelf. [ hum ] Je gaat daar geen grote verwachtingen

...

([76], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_17-21_text.txt'], ["Ja . Oke , nog een keer . Ik ga even terug en vanaf eh Auschwitz hebben , hebben pas de herdenking gehad , eh dat was eh , eh 18 januari ' 44 , gingen we naar eh , dan kregen we de dodentocht : 10 dagen en die bracht ons , tenminste een deel , want de rest viel , er vielen zoveel af fft onder onderweg , kwamen kwam ik in Dachau . Daar heb ik eh , nou ja van de bijzondere ziek daar enzovoort enzovoort , zal ik maar niet vertellen . Even . En vandaar zijn we de paar maanden geweest . We , tenminste de overblijvers . En toen gingen de Joden van Dachau gingen weer op transport en dat transport eh dat was een trein die eh die daar eh op ons te wachten stond en daarvandaan eh , ook onder eh grote moeilijkheden natuurlijk , zijn we heen en weer gereeën . \n"])
++++
0
([76], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_17-21_text.txt'], ["Ja . Oke , nog een keer . Ik ga even terug en v

([80], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_361-367_text.txt'], ["Ja . Eh , wat waren uw ervaringen in die met die taak als ordedienstlid ? Nou , op een gegeven ogenblik dat we de mensen naar eh , na een hele tijd was er weer een transport van vanuit Vught naar Westerbork . Dan hebben we ze naar de naar de dinge gebracht , naar de [ trein ] naar de bussen die ze naar de trein brachten. [ hum ] Nou en eh dan kwamen we daar binnen en dan was ' t eh , eh jullie komen ook in de aan de beurt , jullie komen ook in Polen terecht . Nou ja , inderdaad . Dat werd tegen u gezegd ? Ja ja . Je werd niet eh in dank eh , ze waren hoogstens jaloers dat eh dat wij nog bleven . Ja . Eh , veranderde de de sfeer in het kamp naarmate er een transport aanstaande was ? \n"])
++++
0
([80], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_361-367_text.txt'], ["Ja . Eh , wat waren uw ervaringen in die met die taak als ordedienstlid ? Nou , op een gegeven ogenblik dat we

([85], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_265-276_text.txt'], ["Ja . Was dat agressief ? Kwam dat dreigend op u over ? Eigenlijk niet . Nee . En het typische was dat alles eh buiten de schouwburg hel verlicht was , was eh gewoon daglicht , zo zo sterk was alles verlicht . Ja . En we zijn , we gingen in de tram en we werden ge en dus En zo , zo ik ook zeg dus met felicitaties en een pakje brood gingen we naar Vught . Ja . Uhm . Kunt u zich om romantisch te zijn Ja , ja . Ja ja . Uhm , kunt u zich de treinreis nog voor de geest halen ? Jawel , d' r zaten tegenover me zaten een hele jonge man en vrouw , familie van m' n buren beneden waren dat . En die zaten daar een beetje te vrijen en een beetje verderop zaten de wachtposten , want ' t was eh een trein met coup eh [ coupes ? ] coupes en eh en eh die zaten met eh SS-gevallen , tee hoe heet ' t ? \n"])
++++
0
([85], ['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_265-276_text.txt'], ["Ja . Was 

([93], ['GV_VU_gereformeerden_10_conversation_clipped_150_paragraph_236-241_text.txt'], ['En toen Veenhof dus predikant in Utrecht was , vanwege ook mede die relatie , ben ik toen naar hem toegegaan om daarover te praten met hem van : hoe moet dat. weet u nog wanneer dat ongeveer is , qua jaar ? eh ja dat is in \' 43 geweest . \' 43 . Dus dat is nog voor de vrijmaking . A ja , ja , jaja . Ruim voor de vrijmaking , jazeker. overigens vertelt u iets heel interessants , meneer Alberts . U weet , er wordt altijd gezegd dat ehm , K. Schilder , juist bij Veenhof , ná augustus 1944 begon , bij de aarzelende Veenhof , nogal aandrong : jij moet een keuze maken vriend . Terwijl u nu vertelt , dat vind ik heel boeiend ja , u zegt " Veenhof kwam in huize Alberts , vol vuur , zeggen : \' jullie moeten die richting in . \' " ja. maar dat is u bekend ? \n'])
++++
0
([93], ['GV_VU_gereformeerden_10_conversation_clipped_150_paragraph_236-241_text.txt'], ['En toen Veenhof dus predikant in Utrecht was , 

([97], ['GV_VU_gereformeerden_10_conversation_clipped_150_paragraph_205-211_text.txt'], ['Mensen van de kustplaatsen die moesten daar weg van de Duitsers op een bepaald moment . En toen is die familie , ehm bij mijn tante in Doorn gekomen en die familie die had drie kinderen . En die drie kinderen , die moesten les krijgen . En die hebben toen huisonderwijs gekregen . Ik hoorde dat via mijn tante , en ik heb daar dus een aantal mensen een aantal maanden aan die kinderen op de boerderij in Doorn lesgegeven . Later hoorde ik dat er een andere familie was , die woonde op de Ruiterberg de Ruiterberg dat was een " buiten " , ehm ... Tussen Doorn en Leersum , midden in het bos , en daar woonde een familie Kramer . Dat was een , zeg maar een deftige familie , die ook schitterend buiten woonden . Hadden drie dochtertjes , die dochtertjes die mochten niet naar een gewone school , moesten naar een kostschool toe . \n'])
++++
0
([97], ['GV_VU_gereformeerden_10_conversation_clipped_150_paragraph_2

In [16]:
ranks



[1,
 0,
 0,
 17,
 0,
 233,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 570,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 4,
 0,
 0,
 0,
 0,
 14,
 0,
 13,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 129,
 0,
 0,
 0,
 0,
 31,
 0,
 0,
 81,
 1,
 98,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 26,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2]

### Test fragments 1 & 2 are self-composed 3 is an actual fragmnent from the corpus

In [17]:
test_term = 'hij sloeg er op dat het kapot ging en schoot met zijn geweer. het was verschrikkelijk om aan te zien. ik verstopte me, maar kon het geluid van de explosies niet uit mijn ooren krijgen. Toen het over was ben ik zo snel ik kon gaan rennen.'
test_term_2 = 'Toen waren wij dus in Auschwitz. En toen? Nou, direkt naar een of ander gebouw toe. Uitkleden en de gevangenis kleding aan doen. Ik zag ook meteen al hoe een jongetje van misschien 4 of 5 van zijn moeder weg getrokken werd door een officier van de SS. En maar huilen en gillen.'#' Totdat een schot viel en alles stil was.'
test_term_3 = "Een razzia . Dus die ... . 'Ja , wat doen we ? ' . En we lopen over het dak , en toen hoorden we vanaf de graansilo , daar stond een luchtafweergeschut , en daar kregen we vuur van . Toen vlogen de kogels om onze oren . Toen zij we als de sodomieter via de schoorsteen , in dekking naar Dordtselaan 98 gevlucht . En daar kwamen we en ja , wat doen we ? Ja , wij gaan niet naar beneden toe . Dus wij gingen niet naar beneden toe . Wij gingen perse niet naar beneden ."

process test fragments as corpus

In [18]:
procs_test_term = [token for token in gensim.utils.simple_preprocess(test_term,deacc=True) if token not in dutch_stopwords]
procs_test_term_2 = [token for token in gensim.utils.simple_preprocess(test_term_2,deacc=True) if token not in dutch_stopwords]
procs_test_term_3 = [token for token in gensim.utils.simple_preprocess(test_term_3,deacc=True) if token not in dutch_stopwords]

infer vectors for test fragments

In [19]:
inf_vector_test_term = model.infer_vector(procs_test_term)
inf_vector_test_term_2 = model.infer_vector(procs_test_term_2)
inf_vector_test_term_3 = model.infer_vector(procs_test_term_3)

The above is unstable. infer vectors as median of 100 inferral instances

In [20]:
infarr1 = []
infarr1_cosvar = []
infarr2 = []
infarr3 = []

for i in range(100):
    inf1 = model.infer_vector(procs_test_term)
    inf1_2 = model.infer_vector(procs_test_term)
    inf2 = model.infer_vector(procs_test_term_2)
    inf3 = model.infer_vector(procs_test_term_3)
    cosvar = 1 - sp.spatial.distance.cosine(inf1,inf1_2)
    infarr1.append(inf1)
    infarr2.append(inf2)
    infarr3.append(inf3)
    infarr1_cosvar.append(cosvar)
    
med_infarr1 = np.median(np.array(infarr1),axis=0) #.mean(axis=0)
mean_infarr1 = np.array(infarr1).mean(axis=0)
std_infarr1 = np.array(infarr1).std(axis=0)
mean_infarr1_cosvar = np.array(infarr1_cosvar).mean(axis=0)
med_infarr2 = np.median(np.array(infarr2),axis=0) #.mean(axis=0)
mean_infarr2 = np.array(infarr2).mean(axis=0)
std_infarr2 = np.array(infarr2).std(axis=0)
mean_infarr3 = np.array(infarr3).mean(axis=0)
med_infarr3 = np.median(np.array(infarr3),axis=0)#.mean(axis=0)
std_infarr3 = np.array(infarr3).std(axis=0)

#print(mean_infarr1)
#print(model.infer_vector(procs_test_term,steps=100))
print(med_infarr1)
print(std_infarr1)
print(mean_infarr1_cosvar)
print('----')
print(mean_infarr3)
print(med_infarr3)
print(std_infarr3)
print('++++')
print(inf_vector_test_term_3)
print(inf_vector_test_term)

[ 0.0578131   0.01846248  0.0350742  -0.15228857 -0.20115665 -0.09613059
 -0.1082356   0.14334871 -0.1344417   0.04406869  0.00643789  0.00326736
 -0.09430224  0.0036211  -0.09696069  0.06734325  0.05286681 -0.07858977
  0.01149065 -0.17203829 -0.08368139  0.18595159 -0.1792253   0.09202787
 -0.05758254  0.22699057  0.04550096 -0.07604276 -0.05709763 -0.06327561
  0.15868342  0.00912271  0.07313037  0.05414215  0.08219817  0.07064903
 -0.03426903 -0.20934527  0.01358242  0.06734784  0.05626319  0.05814306
  0.08152154  0.01156388 -0.06382643 -0.02201217 -0.07255004 -0.12080983
 -0.06663983 -0.1815878 ]
[0.07453149 0.07003015 0.06083444 0.07691884 0.06324917 0.07980218
 0.07986313 0.06457207 0.09822506 0.05647783 0.05793546 0.06156718
 0.06110959 0.07131223 0.07152145 0.07246982 0.06868486 0.0744962
 0.05468341 0.07701129 0.05594214 0.06014665 0.05877585 0.08172107
 0.06343749 0.08379988 0.06687865 0.09179297 0.0584568  0.06202474
 0.06518873 0.05630445 0.07156568 0.05709009 0.06060703 

Get similar document vectors as initial test

In [1896]:
#sims = model.docvecs.most_similar([inf_vector_test_term], topn=len(model.docvecs))
#sims_2 = model.docvecs.most_similar([inf_vector_test_term_2], topn=len(model.docvecs))
#sims_3 = model.docvecs.most_similar([inf_vector_test_term_3], topn=len(model.docvecs))

sims = model.docvecs.most_similar([med_infarr1], topn=len(model.docvecs))
sims_2 = model.docvecs.most_similar([med_infarr2], topn=len(model.docvecs))
sims_3 = model.docvecs.most_similar([med_infarr3], topn=len(model.docvecs))


In [1897]:
#corp[0].words
#test = model.docvecs.similarity(corp[0].words,corp[0].words)
#print(test)
#sp.spatial.distance.cosine(test, model.docvecs[0])
#ivtt_12 = np.copy(inf_vector_test_term)
#type(inf_vector_test_term)
#model.save('/Users/eslt0101/Projects/EviDENce/ML/model_default_v50_mc2_e30')

In [1898]:
for i in range(200) :
    print(sims[i])
    if i == 19 :
        print('===================')

(13946, 0.7656733989715576)
(7015, 0.7162730693817139)
(17275, 0.6712950468063354)
(15466, 0.6686472296714783)
(4552, 0.6655747890472412)
(253, 0.6645031571388245)
(6380, 0.6575385332107544)
(1214, 0.6560007929801941)
(14020, 0.6514973640441895)
(14569, 0.6488239169120789)
(22396, 0.6468067169189453)
(16481, 0.6448442339897156)
(2678, 0.6439725756645203)
(19753, 0.643083393573761)
(18036, 0.6343282461166382)
(4114, 0.6340672969818115)
(24432, 0.6312437057495117)
(14945, 0.6298571825027466)
(18728, 0.6298070549964905)
(8768, 0.6296464204788208)
(15229, 0.6286337375640869)
(21305, 0.6268600821495056)
(21388, 0.6243597865104675)
(19807, 0.6206603050231934)
(20106, 0.6201635599136353)
(3587, 0.6197783946990967)
(514, 0.6194276213645935)
(26710, 0.6193383932113647)
(20606, 0.6175340414047241)
(22398, 0.6167048215866089)
(23085, 0.6159239411354065)
(12148, 0.6157993078231812)
(5555, 0.6143925189971924)
(4583, 0.6124187707901001)
(3445, 0.6115472912788391)
(20424, 0.6099816560745239)
(18340, 

In [1899]:
#print(model.docvecs.similarity_unseen_docs(model,corp[0].words,corp[0].words))
#v1 = model.infer_vector(corp[0].words)
#v2 = model.infer_vector(corp[0].words)
#print(np.sqrt(1. - (sp.spatial.distance.cosine(v1,v2))**2))
#print(1. - (sp.spatial.distance.cosine(v1,v2)))
#sp.spatial.distance.cosine(mean_infarr1,model.docvecs[13946]) + 0.764286994934082

In [1900]:
for i in range(25):
    id = sims[i][0]
    print(corp_lookup[id])
    print('+++++++++++++')

([13946], ['GV_DeJager_ReisvandeRazzia_interview_Woudstra_conversation_clipped_150_paragraph_184-189_text.txt'], ['Ik zat dus zo , en kon wel ... ja , van slapen kwam het niet maar je viel toch wel eens weg natuurlijk ... en ja ... je onderging het . Er viel niet veel meer te zeggen , je hoorde een hoop gekerm en een hoop nare dingen eigenlijk , want je ... ja de één houdt zich goed en de ander gaat lopen janken en schreeuwen en ... wordt crazy ervan . Dus je maakte toch wel wat taferelen mee maar dat hoorde je om je heen , want je zat en je moest blijven zitten , ik kon niet anders . Ja ? En ... nou ja .. wat ik al vertelde dus , we hebben die dag doorgereden , die dag doorgereden , we kwamen ergens in de buurt van Berlijn . Ik zat voor het raam , dus je kon het allemaal wel zien . \n'])
+++++++++++++
([7015], ['GV_DeJager_ReisvandeRazzia_interview_Oldenhof_conversation_clipped_150_paragraph_190-194_text.txt'], ['En toen waren we in Assen in de Groningerstraat , daar is toen heel zwaa

reinfer corpus vectores on fully trained and froven model. these will be used for further analysis

In [1889]:

#%time reinf_corp_vec = [ model.infer_vector(c.words) for c in corp]

reinf_corp_vec_med = np.ndarray((100,len(corp),50))
i=0

for j in range(100):
    for i in range(len(corp)):
        c = corp[i]
        reinf_corp_vec_med[j][i][:] = model.infer_vector(c.words)
        i+=1
    j+=1


    
#print(aa)```````````````````````````````````````````````````````
print('=========')
print(np.median(reinf_corp_vec_med,axis=0))


[[[-4.10579592e-01  3.39759916e-01  5.79854175e-02 -2.57429928e-01
   -1.26659885e-01 -8.41341197e-01  2.52375811e-01 -1.83680922e-01
   -2.36400962e-01  1.94103986e-01 -2.95914531e-01  3.92725654e-02
    1.14166044e-01 -4.18560624e-01  2.47841273e-02  5.11171043e-01
    4.41286474e-01 -2.03772277e-01  4.50484827e-02 -6.14511192e-01
    2.17792913e-01  7.01538399e-02  5.37182949e-02 -1.85684562e-02
   -1.42216504e-01  6.53969824e-01 -7.21162558e-02 -7.85734594e-01
   -2.45210007e-01 -2.64831811e-01 -7.30622653e-03 -2.78339505e-01
    7.13209212e-01 -5.99547446e-01  2.10502431e-01  2.32778862e-01
    8.93953517e-02 -2.27496773e-01 -1.21165469e-01 -2.64457643e-01
   -2.68141478e-01  2.60795921e-01  2.84426540e-01 -1.42316923e-01
   -2.63948053e-01 -3.89828384e-01 -1.89370543e-01  9.92087424e-02
   -5.83036765e-02 -4.10142094e-01]
  [-4.87587929e-01  3.39225680e-01 -4.21043523e-02  8.63742083e-03
   -1.93967596e-01 -5.01342773e-01  1.13893613e-01  2.15494037e-02
    7.90233389e-02  7.0763

In [1916]:
reinf_corp_medvec = np.median(reinf_corp_vec_med,axis=0)
print(reinf_corp_medvec)
#print(np.median(reinf_corp_vec_med,axis=0).shape  )  

[[-0.34251951  0.52918094  0.0650135  ...  0.31043735 -0.1035799
  -0.3434217 ]
 [-0.28651591  0.3921694   0.11584316 ...  0.12913995 -0.12679765
  -0.18873925]
 [-0.14764674  0.30281365  0.11900603 ...  0.2351317   0.07376267
  -0.15956078]
 ...
 [-0.09300769  0.43088813 -0.18835094 ... -0.13740871 -0.04644159
  -0.13512794]
 [-0.30387661  0.31770231 -0.03013355 ...  0.07130665 -0.00204581
  -0.46188374]
 [-0.33581319  0.05694224  0.07295781 ...  0.10489847 -0.11573624
  -0.00200729]]


In [12]:
#len(reinf_corp_vec)
#np.save('reinferred_corpus_terms_para_150_medvec.npy',reinf_corp_medvec)

reinf_corp_medvec= np.load('/Users/eslt0101/Projects/EviDENce/ML/reinferred_corpus_terms_para_150_medvec.npy')



Define cosine distance function to use with set of reinferred vectors

In [1947]:
cosdist = np.array([1. - sp.spatial.distance.cosine(med_infarr1,rinfc) for rinfc in reinf_corp_medvec])

#cosdist.max()
b=np.argsort(cosdist)[::-1]
print(b[0:25])
print(cosdist[b[0:25]])
len(b)

[22396 15466 13946  6380 20106 21388 26710  7015  4114 15229 14945 20960
 19753 11824  4552 14020 19807 13121 12716 10385 20424 20311 17276   253
  4555]
[0.74763639 0.74489647 0.7344723  0.73387126 0.71580844 0.71351476
 0.70930658 0.70856002 0.70827753 0.7037911  0.69618466 0.68980258
 0.68718698 0.68598767 0.68534872 0.68514493 0.68427031 0.68426242
 0.67903469 0.67768183 0.6773358  0.67314816 0.67116268 0.67034642
 0.66888773]


27065

In [21]:
cosdist_3 = np.array([1. - sp.spatial.distance.cosine(med_infarr3,rinfc) for rinfc in reinf_corp_medvec])

#cosdist.max()
cc=np.argsort(cosdist_3)[::-1]
print(cc[0:25])
print(cosdist_3[cc[0:25]])
len(cc)

[  461 17257  7601 10555  7317 25328  3984 18839  6179 20630 13946  5314
 15099  4114 19743  3641  6946  7015 20606 22396 18134 11982 14740 15688
  2512]
[0.8583512  0.80742886 0.79974203 0.79643404 0.79201844 0.79132164
 0.78794853 0.77669586 0.77463658 0.77428467 0.76958875 0.769481
 0.76764174 0.76650124 0.76414015 0.76324612 0.75892533 0.75774542
 0.75606606 0.75514318 0.75454964 0.75405139 0.75245088 0.75065691
 0.74923464]


27065

In [22]:
def select_relevant(idlist,corp_lookup):
    selected= []
    for j in range(len(idlist)):
        jj = idlist[j]
        print('---------Snippet '+str(jj)+'---------')
        print(corp_lookup[jj])
        print('------------')
        print('Is this snippet relevant? [y/n]')
        rel = input()
        if rel == 'y' :
            selected.append(jj)
        else :
            pass
        
    return selected

In [23]:
print('The 25 most similar snippets will be displayed.')
print(' Please select those you deem relevant. Are you ready to continue? [please select Yes/No]')
answer = input()

The 25 most similar snippets will be displayed.
 Please select those you deem relevant. Are you ready to continue? [please select Yes/No]
Yes


In [25]:
relevant3=[]
if answer == 'Yes':
    relevant3 = select_relevant(cc[0:24],corp_lookup)
    #for i in range(25):
    #    ii = b[i]
    #    print('---------Snippet '+str(ii)+'---------')
    #    print(corp_lookup[ii])
    #    print('------------')
    #    print('Is this snippet relevant? [y/n]')
    #    rel = input()
    #    if rel == 'y' :
    #        relevant.append(ii)
    #    else :
    #        pass
else :
    print('No relevant snippets have been selected.')

---------Snippet 461---------
([461], ['GV_Museon_Jochem Knoester_conversation_clipped_150_paragraph_590-593_text.txt'], ['Je werd wel eens achterna gezeten , maar ze kregen je bijna niet want we waren echt geveltoeristen o , net als o-over de nokken van de daken liepen we , in de schuine daken zo , weet je wel . Want , want , ja , we haalden van alles uit . Ja , ja . " " Oké , ik wil u bedanken voor het gesprek . " " Ja , graag gedaan . " \n'])
------------
Is this snippet relevant? [y/n]
n
---------Snippet 17257---------
([17257], ['GV_Schiedam_gastgezin1_03_conversation_clipped_150_paragraph_702-711_text.txt'], ["Ja , toen werd ik in mijn kippennek gepakt ! Nou , die Duitsers hebben mij knap bang gemaakt . Ze zeiden toen ' we gaan je ouders ophalen ' en weet ik wat . Een hele reprimande heb ik gekregen . De angst zat er goed in , ik heb het ook nooit meer gedaan . Ik ben niet meer eten gaan stelen . Ja , ik heb het wel eens geprobeerd , maar ik durfde niet . U durfde niet . Ik kan m

y
---------Snippet 13946---------
([13946], ['GV_DeJager_ReisvandeRazzia_interview_Woudstra_conversation_clipped_150_paragraph_184-189_text.txt'], ['Ik zat dus zo , en kon wel ... ja , van slapen kwam het niet maar je viel toch wel eens weg natuurlijk ... en ja ... je onderging het . Er viel niet veel meer te zeggen , je hoorde een hoop gekerm en een hoop nare dingen eigenlijk , want je ... ja de één houdt zich goed en de ander gaat lopen janken en schreeuwen en ... wordt crazy ervan . Dus je maakte toch wel wat taferelen mee maar dat hoorde je om je heen , want je zat en je moest blijven zitten , ik kon niet anders . Ja ? En ... nou ja .. wat ik al vertelde dus , we hebben die dag doorgereden , die dag doorgereden , we kwamen ergens in de buurt van Berlijn . Ik zat voor het raam , dus je kon het allemaal wel zien . \n'])
------------
Is this snippet relevant? [y/n]
y
---------Snippet 5314---------
([5314], ['GV_Verhalis_kloosterzusters_08b_conversation_clipped_150_paragraph_165-166_te

y
---------Snippet 22396---------
([22396], ['GV_DeJager_ReisvandeRazzia_interview_Breda_conversation_clipped_150_paragraph_392-398_text.txt'], ['En daar die hebben ons opgevangen en toen hebben we met al die mensen op de grond geslapen . Nou , en toen zijn we weer gaan lopen en toen kwamen we bij een tante . Ik mocht bij mijn ene tante en mijn vader en moeder mochten bij de andere tante . Nou , dat zo is het allemaal gegaan . Dus dat is allemaal ... Nou , en toen dacht je " het is voorbij " , maar dan hoorde je iedere keer weer \' s nachts eh , maar ja , wij hadden geen huis en mijn vader en moeder , die hadden het ook natuurlijk niet naar de zin om bij een ander op een schopstoel te zitten . Dus op den duur konden wij in de Rozenstraat wonen . Daar kregen we een woning . Maar dat wou mijn moeder niet , want daar waren de ramen allemaal kapot en schieten . \n'])
------------
Is this snippet relevant? [y/n]
y
---------Snippet 18134---------
([18134], ['GV_Nijkerk_Vluchtelingen_06_conve

The below has been run several times in an interactive fashion to select and manually iterate over the selection process. This is very muchg still in a bleeding edge development state.

In [26]:
print(relevant3)

[25328, 3984, 18839, 20630, 13946, 19743, 3641, 6946, 7015, 20606, 22396, 15688]


In [63]:
print(len(corp_lookup))
ph =(((np.random.rand(25)*27065).round()))
print(ph)
phi = [int(pp) for pp in ph]
print(phi)

27065
[14885. 22782. 24532. 22907. 17583. 23759. 24034. 24020. 15968. 20226.
 13021. 21523. 19059. 14766. 14151. 21991. 18545. 10512.  7486.   497.
 26654.  8026.  7332. 11912. 20101.]
[14885, 22782, 24532, 22907, 17583, 23759, 24034, 24020, 15968, 20226, 13021, 21523, 19059, 14766, 14151, 21991, 18545, 10512, 7486, 497, 26654, 8026, 7332, 11912, 20101]


In [64]:
relevantrand=[]
if answer == 'Yes':
    
    
    
    relevantrand = select_relevant(phi,corp_lookup)
    #for i in range(25):
    #    ii = b[i]
    #    print('---------Snippet '+str(ii)+'---------')
    #    print(corp_lookup[ii])
    #    print('------------')
    #    print('Is this snippet relevant? [y/n]')
    #    rel = input()
    #    if rel == 'y' :
    #        relevant.append(ii)
    #    else :
    #        pass
else :
    print('No relevant snippets have been selected.')

---------Snippet 14885---------
([14885], ['GV_Zigma_zeemansgezin_02_conversation_clipped_150_paragraph_861-870_text.txt'], ['Terwijl de dollar toen op een rijksdaalder stond . Dus dat was krap achthonderd gulden . Dat was negen maanden maandbrief voor mijn moeder . Dat zijn de dingen die ga je je dan afvragen . Hoe zit dat ? Wat , wat hè , ja . Hier nog veel meer . Dat is in april . Nog steeds opnemen , opnemen , opnemen . Ja . En in mei hoort hij pas dat wij nog leven . Want hij was in New York toen de oorlog afliep . Ja En wat staat er in het zakboekje , dat kleine zakboekje Dat is een beetje cryptisch . Daar staan alleen maar telefoonnummers in die mij niks zeggen in New York . Ja U heeft nooit geprobeerd te achterhalen van wie die telefoonnummers waren Nee , dat kan niet meer . Dat was te laat . Die bestaan niet meer . Ja . Maar het vervelende is . \n'])
------------
Is this snippet relevant? [y/n]
n
---------Snippet 22782---------
([22782], ['GV_Wieberdink_dodenmars_01a_conversat

y
---------Snippet 20226---------
([20226], ['GV_Zigma_koopvaardij_07_conversation_clipped_150_paragraph_68-75_text.txt'], ["Dus iedereen ... en de patrijspoorten die moesten zwart gemaakt worden . Afijn het was allemaal donker en iedereen moest zijn steentje bijdragen . En dat ging zo door En u heeft ook uw steentje bijgedragen Ja ik heb ook geschilderd . Iedereen moest zijn eigen eten halen . Anders moesten de lichtmatrozen , ik was lichtmatroos , het eten voor de bemanning halen maar dat hoefde niet meer . Dat ging zo door en er waren protesten natuurlijk . 'Waarmee moeten wij vechten ? We hebben niks . We hebben geen revolver , geen kanon , geen mitrailleur ' . En toen zei die koksmaat , want de kapitein was er ook bij : ' We willen wel varen , we willen Nederland bevrijden , maar dan moet je wel terug kunnen vechten Er komen allemaal vliegtuigen aan , wat moet je dan doen ? ' . En toen zei die koksmaat : ' Kapitein , een katapult ' ( gelach ) . Die kreeg een draai om zijn oren . \

n
---------Snippet 7486---------
([7486], ['GV_GAR_bombardement_02_conversation_clipped_150_paragraph_69-77_text.txt'], ["Want zo' n Katendrecht was prachtig , ik heb er dan foto 's van in de krantjes gezien , van die draken hadden ze allemaal gemaakt , een hele toestand is dat toen geweest . Die Chinese feesten zeg maar . Ja een Chinees feest was dat . En dan mochten ze er mee naartoe en ik was de derde en ik mocht niet mee want ik was maar een meisje En dat gold voor uw zussen ook of mochten die later meer dan u ? Nou toen was mijn vader eigenlijk al een beetje aan het afzakken . En ja die mochten ook niet mee hoor want het was altijd alleen de jongens maar . U vertelde dat uw vader die rijwielhandel weg had moeten doen , wat heeft hij erna zoal gedaan ? Nou toen is hij voor mijn grootvader gaan werken . Heeft hij wat schilderwerk gedaan . \n"])
------------
Is this snippet relevant? [y/n]
n
---------Snippet 497---------
([497], ['GV_Museon_Jochem Knoester_conversation_clipped_150_pa

In [65]:
print(relevantrand
     )

[22907, 17583, 15968, 21991, 18545]


In [1943]:
cosdist_rel1 = np.array([1. - sp.spatial.distance.cosine(reinf_corp_medvec[20106],rinfc) for rinfc in reinf_corp_medvec])

In [1944]:
b2=np.argsort(cosdist_rel1)[::-1]
print(b2[0:25])
print(cosdist_rel1[b2[0:25]])

[20106 20502 13423  3152 10415 13946  1263 24390   253 18340 15093 17276
 24378  5603     2  3437  4207 20153 17197 25237  4031  5441 18036  8439
 26618]
[1.         0.82115026 0.81513107 0.80872781 0.80057436 0.79939413
 0.79738701 0.79701688 0.79668153 0.79606148 0.7952796  0.79236752
 0.79106498 0.78929426 0.78878198 0.78850495 0.78389686 0.78274665
 0.78188818 0.78153604 0.78082591 0.77966812 0.779103   0.77836438
 0.77824621]


In [1951]:
relevant2=select_relevant(b2[0:25],corp_lookup)

#for j in range(25):
#    jj = b2[j]
#    print
#    print(corp_lookup[jj])
#    print('-------------')



---------Snippet 20106---------
([20106], ['GV_AmsterdamNoord_Fokker_07_conversation_clipped_150_paragraph_94-102_text.txt'], ["En ik had eh , ik had een onderjurkje aan , weet ik nog wel , en een truitje . En voor de rest had ik niks aan . Misschien ook nog wel een onderbroekje , dat zal wel . Maar geen schoenen of kousen , of zo. Verder helemaal niks . Nou , en eh , we liepen daar naar toe en stonden in de gang en toen gebeurde het . En ja , dat is iets wat ik , dat zal ik nooit meer uit m' n eh , uit m' n ... systeem krijgen , denk ik . Nee , dat is , dat is zoiets vreselijks . Ineens dan , ja , dan komt er ga , een donderend geweld , wat je nooit eerder hebt gehoord . Zo vreselijk , en alles werd zwart . En eh , toen heeft m' n moeder . En alles begon naar beneden te gg , vallen kalk en zo. \n"])
------------
Is this snippet relevant? [y/n]
y
---------Snippet 20502---------
([20502], ['GV_DeJager_ReisvandeRazzia_interview_Aart_den_Haan_conversation_clipped_150_paragraph_249-256_tex

y
---------Snippet 15093---------
([15093], ['GV_DeJager_ReisvandeRazzia_interview_Snel_conversation_clipped_150_paragraph_230-237_text.txt'], ["Echt niet pakje aan , deftig , nee , in ' 45 naar kantoor toe . Ja , wat nog ... En heeft u de .. , de stad zien opbouwen ? Ja , uiteraard , nog , ze zijn nog bezig . Maar ja , er is ook niks aan , aan die stad . Ja , qua bouwstijl is het een prachtige stad natuurlijk met al die gebouwen , maar toen ben ik naar de Rotterdamse Bank gegaan . Toen nog de Rotterdamse Bank . Daar is de AMRO Bank bij ge - komen , de ABN bij gekomen , maar toen was het echt nog de Rotterdamse Bank op de Cool - singel of wat dan ook , daar ben ik dus , ja , daar heb ik mijn maagzweer geloof ik opgelopen . Dat was een leventje eigenlijk , verschrikkelijk . Dat was ' s morgens bij wijze van half 9 beginnen , maar 5 voor half 9 je stempel op je stempel .. , op je stempelkaart natuurlijk . \n"])
------------
Is this snippet relevant? [y/n]
n
---------Snippet 17276--------

y
---------Snippet 4031---------
([4031], ['GV_DeJager_ReisvandeRazzia_interview_Dorsman_conversation_clipped_150_paragraph_617-620_text.txt'], ['Nou nee , later heb ik nog wel eens gedacht , ik denk " verdorie eigenlijk ! " , maar ja , dan hoorde je weer zoveel dat er zoveel is .. , er is zoveel erger gebeurd dan ik , ik heb het overleefd " . Ik ben mijn wintertenen kwijtgeraakt , daarom noem ik het altijd " wintertenen " , en daar met al die natte voeten en die narig-heid in die kou , dat ben ik daar kwijtgeraakt . Dus en nog steeds . Alleen , ik heb wel last van trombose gekregen later . En ik heb het echt 14 , 15 jaar geleden aan mijn hart gekregen . Een hartinfarct . Maar ik heb zo\' n ding heb ik in mijn hart en ik werk .. , ik werk in mijn tuintje , je hebt misschien heel gauw mijn tuintje gezien . En ik fiets en ik wandel . En we gaan verder . \n'])
------------
Is this snippet relevant? [y/n]
n
---------Snippet 5441---------
([5441], ['GV_Verhalis_kloosterzusters_07a_conversat

In [1953]:
print(relevant2)

[20106, 10415, 13946, 1263, 24390, 18340, 17276, 24378, 2, 4207, 20153, 17197, 25237, 5441, 8439, 26618]


In [67]:
relevant2 = [20106, 10415, 13946, 1263, 24390, 18340, 17276, 24378, 2, 4207, 20153, 17197, 25237, 5441, 8439, 26618]

In [70]:
reit_r2 = reinf_corp_medvec[relevant2].mean(axis=0)

In [71]:
cosdist_reit_r2 = np.array([1. - sp.spatial.distance.cosine(reit_r2,rinfc) for rinfc in reinf_corp_medvec])

In [74]:
r2=np.argsort(cosdist_reit_r2)[::-1]
print(r2[0:25])
print(cosdist_reit_r2[r2[0:25]])

[ 3437  1263 17197  8479 18175 24432 20741  4243 20502  2312 25843  5441
  2165  8428 11518  1200 23775 24993 22090 14465  8471 10415 10969 18558
   928]
[0.9424102  0.93936797 0.93254753 0.93221097 0.93212913 0.92935743
 0.928519   0.92803785 0.92761638 0.92743479 0.92724534 0.92719832
 0.92534004 0.92262524 0.92236441 0.9218516  0.92151838 0.91880438
 0.9169306  0.9168689  0.91664595 0.91655519 0.91655425 0.91553688
 0.91478988]


In [75]:
relevant_r2=select_relevant(r2[0:25],corp_lookup)

---------Snippet 3437---------
([3437], ['GV_DeJager_ReisvandeRazzia_interview_Scheffers_conversation_clipped_150_paragraph_168-173_text.txt'], ['En dan was het , ja , een stukje .. , een stukje .. , meestal dat zure Duitse Köch wat je kreeg , zo\' n .. , zo\' n een paar centimeter dik , weet je wel . En dat kreeg je dan voor die dag . Nou , en dan werd het , je weet het wel , in de winteravond is het vroeg donker , dus dan .. , dan lag je daar en ja , verder niks . En wel hoorde ik ineens op een dag , op een nacht , op een avond hoorde ik er één zeggen zo uit het donker " potverdikkeme " zegt hij , " het is al 26 maart ! " , weet je wel . En ik zeg " 26 maart ? " . Ik zeg " dan ben ik jarig geweest vandaag ! " . Dat was mijn 18de verjaardag . Ja , de 18de verjaardag . \n'])
------------
Is this snippet relevant? [y/n]
n
---------Snippet 1263---------
([1263], ['GV_Lumen_verzetsgroep_JL_02_conversation_clipped_150_paragraph_46-54_text.txt'], ["En op die manier ben ik toen , eerder in h

y
---------Snippet 25843---------
([25843], ['08_JKKV_2003_KOOS_VALK_conversation_clipped_150_paragraph_437-445_text.txt'], ["Je moest ook op appél . Op appél staan . Ja . In rijen van vijf . Rijen van vijf . Nou , was ook ' t zelfde , van zelfde laken een pak als met die marcheren . Allemaal in gelid staan en eh die niet goed stond die kreeg een pak rammel of een schop of een slag of . Nou ja goed , dan moest er wel eentje piesen , moest er wel eentje poepen , moest er eentje dit , moest er eentje dat . En het waren niet alleen de negenjarigen die er stonden . Nee , d' r was vanaf vier of vijf jaar af , kleine kinderen tot aan mijn leeftijd toe . Want de tienjarigen heb ik daar niet zo gauw gezien , dat weet ik zo niet . Maar ook die kleintjes . Nou die konden natuurlijk ook niet stilstaan en die die gingen ook spelen en doen en er werd maar geteld en er werd maar geteld . \n"])
------------
Is this snippet relevant? [y/n]
y
---------Snippet 5441---------
([5441], ['GV_Verhalis_kloost

y
---------Snippet 8471---------
([8471], ['GV_Groningen_Scholtenhuis_01_conversation_clipped_150_paragraph_665-672_text.txt'], ['Dat vond ik fijn . Hij zei zo is het gekomen . Ik zei maar wat jij mij hebt aangedaan dat was iets . Hij zei sorry het is gebeurd . Het is gebeurd ik heb mijn straf ook gehad geloof ik dus klaar . Ik ben er wel tevreden mee . Hij zei maar ik heb er wel soms last van nog . Ik zei nee , toen zei hij ja , we waren beesten toen , we waren geen mensen . Wij waren beesten . Heeft hij zelf gezegd , toen heb ik hem een hand gegeven . Dus we gaven elkaar een hand , ik zei zo dat houdt in dat je toch weer niet in mijn cafetaria mag komen he Zo werkt dat niet jongeman . Ik zeg geen vriend , ik geef je een hand , alles is vergeten en gedaan . Ik zeg maar je mag niet weer komen . Ik wil je echt niet weer zien . En dat was het . \n'])
------------
Is this snippet relevant? [y/n]
y
---------Snippet 10415---------
([10415], ['GV_AmsterdamNoord_Fokker_05_conversation_clipped

In [76]:
print(relevant_r2)

[1263, 17197, 8479, 24432, 4243, 2312, 25843, 5441, 2165, 1200, 22090, 14465, 8471, 10415, 10969, 928]


In [78]:
reit_r3 = reinf_corp_medvec[relevant_r2].mean(axis=0)

In [79]:
cosdist_reit_r3 = np.array([1. - sp.spatial.distance.cosine(reit_r3,rinfc) for rinfc in reinf_corp_medvec])

In [80]:
r3=np.argsort(cosdist_reit_r3)[::-1]
print(r3[0:25])
print(cosdist_reit_r3[r3[0:25]])

[20741  4243  2312 25843  8479 11518 19759 18411 23494 18367  1200 18558
  6672 19320 16114  5845 10672  2051 25639 10592  5848 18368 15722 25635
  2105]
[0.97141731 0.97045024 0.96924278 0.96820835 0.96634517 0.96485442
 0.96281003 0.96033114 0.9599272  0.95992245 0.95956124 0.95937682
 0.95914327 0.95912276 0.95907872 0.95898674 0.95897769 0.95837528
 0.95832419 0.95821154 0.95820672 0.95818807 0.95813609 0.95804829
 0.95799794]


In [82]:
relevant_r3=select_relevant(r3[0:25],corp_lookup)

---------Snippet 20741---------
([20741], ['GV_Schiedam_gastgezin2_09_conversation_clipped_150_paragraph_1-12_text.txt'], ['Dit is een interview in het kader van " Schiedamse Bleekneusjes " . Een project van het Gemeentearchief Schiedam in het kader van Erfgoed van de oorlog een project dat subsidie krijgt van het ministerie van WVS . Ik ben Caroline Nieuwendijk de interviewer . Het is vandaag 2 december 2009 . Ik zit tegenover meneer Dekker . Wanneer bent u geboren ? Op 11 juli 1935 . U bent niet in Schiedam geboren , hè ? Ik ben niet in Schiedam geboren , in Dordrecht ben ik geboren . Hebt u daar de hele tijd gewoond voordat u naar Schiedam ging ? Ja , in 1940 zijn we naar Schiedam verhuisd , dus ik heb daar een kleine vijf jaar gewoond . In 1939 is uw moeder overleden , hè ? Ja , aan borstkanker . Haar heb ik eigenlijk nooit gekend in de zin dat ik me haar niet voor de geest kan halen . \n'])
------------
Is this snippet relevant? [y/n]
n
---------Snippet 4243---------
([4243], ['08

y
---------Snippet 18367---------
([18367], ['GV_NIOD_Buchenwald_18_clipped_150_paragraph_29-37_text.txt'], ['Naast tewerkstelling in arbeidscommando ´ s , verhuurde de SS gevangenen aan grote ondernemingen . Buchenwald had dus niet alleen zijn eigen positie in het vernietigingsapparaat , maar ook een duidelijke verbinding met de maatschappij en economie . Het gevolg was dat Buchenwald uitgroeide tot één van de grootste nationaalsocialistische kampcomplexen met van de Rijn tot aan de Elbe een netwerk van 136 buitenkampen . Het totale aantal gevangenen in Buchenwald groeide van ruim 11.000 in januari 1943 naar ruim 84.000 in de herfst van 1944 . Ze werkten in onvoorstelbare omstandigheden . Wie niet in staat was om te werken , werd uitgeselecteerd voor een transport naar een vernietigingskamp of stierf door uitputting . In totaal zijn er 250.000 gevangenen geweest waarvan er bijna 60.000 zijn omgekomen . Begin 1945 werd Buchenwald eindstation voor evacuatietransporten uit de opgeheven k

y
---------Snippet 2051---------
([2051], ['GV_NIOD_Buchenwald_17_clipped_150_paragraph_29-37_text.txt'], ['Naast tewerkstelling in arbeidscommando ´ s , verhuurde de SS gevangenen aan grote ondernemingen . Buchenwald had dus niet alleen zijn eigen positie in het vernietigingsapparaat , maar ook een duidelijke verbinding met de maatschappij en economie . Het gevolg was dat Buchenwald uitgroeide tot één van de grootste nationaalsocialistische kampcomplexen met van de Rijn tot aan de Elbe een netwerk van 136 buitenkampen . Het totale aantal gevangenen in Buchenwald groeide van ruim 11.000 in januari 1943 naar ruim 84.000 in de herfst van 1944 . Ze werkten in onvoorstelbare omstandigheden . Wie niet in staat was om te werken , werd uitgeselecteerd voor een transport naar een vernietigingskamp of stierf door uitputting . In totaal zijn er 250.000 gevangenen geweest waarvan er bijna 60.000 zijn omgekomen . Begin 1945 werd Buchenwald eindstation voor evacuatietransporten uit de opgeheven kam

y


In [84]:
print(relevant_r3)

[4243, 2312, 25843, 8479, 18411, 23494, 18367, 1200, 6672, 19320, 5845, 10672, 2051, 25639, 10592, 5848, 18368, 15722, 25635, 2105]
